In [1]:
import os
import numpy as np
import sympy as sp
import torch

In [2]:
cd ../

/home/kpgelvan/SymbolicMathematics


In [3]:
from src.utils import AttrDict
from src.envs import build_env
from src.model import build_modules

from src.utils import to_cuda
from src.envs.sympy_utils import simplify

### 

In [4]:
OPERATORS = {
        # Elementary functions
        'add': 2,
        'sub': 2,
        'mul': 2,
        'div': 2,
        'pow': 2,
        'rac': 2,
        'inv': 1,
        'pow2': 1,
        'pow3': 1,
        'pow4': 1,
        'pow5': 1,
        'sqrt': 1,
        'exp': 1,
        'ln': 1,
        'abs': 1,
        'sign': 1,
        # Trigonometric Functions
        'sin': 1,
        'cos': 1,
        'tan': 1,
        'cot': 1,
        'sec': 1,
        'csc': 1,
        # Trigonometric Inverses
        'asin': 1,
        'acos': 1,
        'atan': 1,
        'acot': 1,
        'asec': 1,
        'acsc': 1,
        # Hyperbolic Functions
        'sinh': 1,
        'cosh': 1,
        'tanh': 1,
        'coth': 1,
        'sech': 1,
        'csch': 1,
        # Hyperbolic Inverses
        'asinh': 1,
        'acosh': 1,
        'atanh': 1,
        'acoth': 1,
        'asech': 1,
        'acsch': 1,
        # Derivative
        'derivative': 2,
        # custom functions
        'f': 1,
        'g': 2,
        'h': 3,
    }

symbols = ['I', 'INT+', 'INT-', 'INT', 'FLOAT', '-', '.', '10^', 'Y', "Y'", "Y''"]

In [5]:
constants = ['pi', 'E']
variables = ['x', 'y', 'z', 't']
functions = ['f', 'g', 'h']
elements = [str(i) for i in range(-10, 10)]
coefficients = [f'a{i}' for i in range(10)]

In [6]:
no_child_symbols = constants + variables + functions + elements + coefficients

###

In [27]:
'10'.isdigit()

True

In [28]:
from tqdm import tqdm
import queue

def get_ancestors(exp_list, exp_len):
    q = queue.LifoQueue()
    q.put(-1)                            # so last element gets this parent but doesn't save it

    ancestors = {0: []}
    node2parent = {}
    levels = {0: -1}

    parent = 0
    for i in range(exp_len):
        op_now = exp_list[i]

        node2parent[i] = parent
        levels[i] = levels[parent] + 1

        if op_now in OPERATORS or op_now in symbols:   # <=> node has children
            if op_now in OPERATORS and OPERATORS[op_now] == 2:    # <=> node has 2 children
                q.put(i)
            parent = i
        elif op_now in no_child_symbols:
            if op_now.isdigit() and i + 1 < exp_len and exp_list[i + 1].isdigit():   # e.x. 18
                parent = i
            else:
                parent = q.get()
        else:
            print(op_now)
            raise(NotFound)
        ancestors[i] = [i] + ancestors[node2parent[i]]

    return ancestors, levels

In [29]:
def get_path(i, j):
    if i == j:
        return "<self>"
    anc_i = set(ancestors[i])
      
    for node in ancestors[j][-(levels[i] + 1) :]:
        if node in anc_i:
            up_n = levels[i] - levels[node]
            down_n = levels[j] - levels[node]
            return str(up_n + 0.001 * down_n)

In [30]:
def get_ud_masks(ancestors, levels, exp_len):
    path_rels = []
    for i in range(exp_len):
        path_rels.append(" ".join([get_path(i, j) for j in range(exp_len)]))
    
    return path_rels

###

In [31]:
F_prefix  = ['mul', 'x', 'tan', 'mul', 'pow', 'x', 'INT-', '1', 'exp', 'x']

In [32]:
ancestors, levels = get_ancestors(F_prefix, len(F_prefix))
rel_matrix = get_ud_masks(ancestors, levels, len(F_prefix))
rel_matrix

['<self> 1 1 2 3 4 4 5 3 4',
 '1 <self> 2 3 4 5 5 6 4 5',
 '1 2 <self> 1 2 3 3 4 2 3',
 '2 3 1 <self> 1 2 2 3 1 2',
 '3 4 2 1 <self> 1 1 2 2 3',
 '4 5 3 2 1 <self> 2 3 3 4',
 '4 5 3 2 1 2 <self> 1 3 4',
 '5 6 4 3 2 3 1 <self> 4 5',
 '3 4 2 1 2 3 3 4 <self> 1',
 '4 5 3 2 3 4 4 5 1 <self>']

###

In [33]:
model_path = '../checkpoint.pth'
assert os.path.isfile(model_path)

In [34]:
params = params = AttrDict({

    # environment parameters
    'env_name': 'char_sp',
    'int_base': 10,
    'balanced': False,
    'positive': True,
    'precision': 10,
    'n_variables': 1,
    'n_coefficients': 0,
    'leaf_probs': '0.75,0,0.25,0',
    'max_len': 512,
    'max_int': 5,
    'max_ops': 15,
    'max_ops_G': 15,
    'clean_prefix_expr': True,
    'rewrite_functions': '',
    'tasks': 'prim_fwd',
    'operators': 'add:10,sub:3,mul:10,div:5,sqrt:4,pow2:4,pow3:2,pow4:1,pow5:1,ln:4,exp:4,sin:4,cos:4,tan:4,asin:1,acos:1,atan:1,sinh:1,cosh:1,tanh:1,asinh:1,acosh:1,atanh:1',

    # model parameters
    'cpu': False,
    'emb_dim': 1024,
    'n_enc_layers': 6,
    'n_dec_layers': 6,
    'n_heads': 4,
    'dropout': 0,
    'attention_dropout': 0,
    'sinusoidal_embeddings': False,
    'share_inout_emb': True,
    'reload_model': model_path,
    'max_relative_pos':0,
    'use_neg_dist':False
})

In [35]:
env = build_env(params)
x = env.local_dict['x']

In [36]:
modules = build_modules(env, params)
encoder = modules['encoder']
decoder = modules['decoder']

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

###

In [37]:
import sys
sys.path.append('src/envs/')
sys.path.append('src')

In [38]:
#from char_sp import prefix_to_infix, infix_to_sympy

In [39]:
#from utils import create_logger
#from utils import bool_flag
#from utils import timeout, TimeoutError
#from char_sp import prefix_to_infix, infix_to_sympy


In [40]:
f_prefix = ['sub', "Y'", 'pow', 'x', 'INT+', '2']
x1_prefix = env.clean_prefix(['sub', 'derivative', 'f', 'x', 'x'] + f_prefix)
x1_prefix
#x1 = torch.LongTensor(
#    [env.eos_index] +
#    [env.word2id[w] for w in x1_prefix] +
#    [env.eos_index]
#).view(-1, 1)
#len1 = torch.LongTensor([len(x1)])
#x1, len1 = to_cuda(x1, len1)


['sub', "Y'", 'sub', "Y'", 'pow', 'x', 'INT+', '2']

In [48]:
env.parse_int(['INT+', '1', '8', 'pow', 'x', 'INT+', '2'])

InvalidPrefixExpression: 'Invalid integer in prefix expression'

In [42]:
env.prefix_to_infix(x1_prefix)

InvalidPrefixExpression: 'Invalid integer in prefix expression'

In [43]:
sp.S(env.infix_to_sympy(env.prefix_to_infix(x1_prefix)))

InvalidPrefixExpression: 'Invalid integer in prefix expression'

In [54]:
import json

for set_name in ['valid', 'test', 'train']:
    with open('data/prim_fwd.' + set_name, 'r') as expressions:
        with open('data/rel_matrix_'+set_name+'.json', 'w') as rel_matrix_json:
            for i, line in tqdm(enumerate(expressions)):
                #print(line)
                q, a = line.split('|')[1].split('\t')
                #print(q, ';', a)
                
                q = q.split()
                a = a.split()
                
                ancestors, levels = get_ancestors(q, len(q))
                rel_matrix_q = get_ud_masks(ancestors, levels, len(q))              

                #ancestors, levels = get_ancestors(a, len(a))
                #rel_matrix_a = get_ud_masks(ancestors, levels, len(a))
                
                rel_matrix_json.write(json.dumps(rel_matrix_q, indent=0))


0it [00:00, ?it/s]
480it [00:00, 4789.45it/s]
848it [00:00, 4391.08it/s]
1195it [00:00, 4062.11it/s]
1527it [00:00, 3806.02it/s]
1836it [00:00, 3557.84it/s]
2122it [00:00, 3291.33it/s]
2406it [00:00, 3133.21it/s]
2690it [00:00, 2956.75it/s]
2967it [00:00, 2380.31it/s]
3210it [00:01, 2095.23it/s]
3430it [00:01, 1938.00it/s]
3633it [00:01, 1840.17it/s]
3825it [00:01, 1768.94it/s]
4008it [00:01, 1647.04it/s]
4179it [00:01, 1614.21it/s]
4348it [00:01, 1632.63it/s]
4516it [00:01, 1646.03it/s]
4683it [00:02, 1610.00it/s]
4849it [00:02, 1623.01it/s]
5013it [00:02, 1621.51it/s]
5176it [00:02, 1601.93it/s]
5337it [00:02, 1587.63it/s]
5511it [00:02, 1630.06it/s]
5675it [00:02, 1622.82it/s]
5838it [00:02, 1624.05it/s]
6002it [00:02, 1628.49it/s]
6170it [00:02, 1641.70it/s]
6335it [00:03, 1617.27it/s]
6498it [00:03, 1617.13it/s]
6666it [00:03, 1634.46it/s]
6830it [00:03, 1613.85it/s]
6992it [00:03, 1595.88it/s]
7152it [00:03, 1590.46it/s]
7329it [00:03, 1639.74it/s]
7494it [00:03, 1602.25it/s]
76

KeyboardInterrupt: 